# 0.8 TD Ameritrade API Streaming Data

As I progress in my research, I come to the conclusion that to accurately model price movements in the market will require tick level data. I have not explored this kind of market data yet, so I should do so. 

Of great interest to me is the streaming functionality available. Currently, I am using TDAmeritrade and mostly constrained to the data they provide. 

Since this is the case, let's explore the streaming API and see what what it can give me. 

In [ ]:
import requests
import urllib
import json
import requests
import dateutil.parser
from datetime import datetime
from splinter import Browser 
import config

In [ ]:
executable_path = {'executable_path':r'/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
method = 'GET'
url = 'https://auth.tdameritrade.com/auth?'
client_code = client_id + '@AMER.OAUTHAP'
payload = {'response_type':'code', 'redirect_uri':'https://127.0.0.1', 'client_id':client_code}

In [ ]:
p = requests.Request(method, url, params=payload).prepare()
myurl = p.url
browser.visit(myurl)

In [ ]:
new_url = browser.url
new_url

In [ ]:
parse_url = urllib.parse.unquote(new_url.split('code=')[1])
parse_url

In [ ]:
browser.quit()

In [ ]:
client_id

In [ ]:
account_number

In [ ]:
url = r"https://api.tdameritrade.com/v1/oauth2/token"

headers = {"Content-Type":"application/x-www-form-urlencoded"}

# define the payload
payload = {'grant_type': 'authorization_code', 
           'access_type': 'offline', 
           'code': parse_url, 
           'client_id':client_id, 
           'redirect_uri':'https://127.0.0.1'}

In [ ]:
authReply = requests.post(r'https://api.tdameritrade.com/v1/oauth2/token', headers = headers, data=payload)

In [ ]:
authReply

In [ ]:
decoded_content = authReply.json()

In [ ]:
access_token = decoded_content['access_token']
headers = {'Authorization': "Bearer {}".format(access_token)}

headers

In [ ]:
def unix_time_millis(dt):
    
    # grab the starting point, so time '0'
    epoch = datetime.utcfromtimestamp(0)
    
    return (dt - epoch).total_seconds() * 1000.0

In [ ]:
endpoint = "https://api.tdameritrade.com/v1/userprincipals"
params = {'fields':'streamerSubscriptionKeys,streamerConnectionInfo'}

# make a request
content = requests.get(url = endpoint, params = params, headers = headers)
userPrincipalsResponse = content.json()

In [ ]:
userPrincipalsResponse

In [ ]:
tokenTimeStamp = userPrincipalsResponse['streamerInfo']['tokenTimestamp']
date = dateutil.parser.parse(tokenTimeStamp, ignoretz = True)
tokenTimeStampAsMs = unix_time_millis(date)

In [ ]:
userPrincipalsResponse = config.TDAMERITRADE_CREDS
credentials = {"userid": userPrincipalsResponse['accounts'][0]['accountId'],
               "token": userPrincipalsResponse['streamerInfo']['token'],
               "company": userPrincipalsResponse['accounts'][0]['company'],
               "segment": userPrincipalsResponse['accounts'][0]['segment'],
               "cddomain": userPrincipalsResponse['accounts'][0]['accountCdDomainId'],
               "usergroup": userPrincipalsResponse['streamerInfo']['userGroup'],
               "accesslevel":userPrincipalsResponse['streamerInfo']['accessLevel'],
               "authorized": "Y",
               "timestamp": int(tokenTimeStampAsMs),
               "appid": userPrincipalsResponse['streamerInfo']['appId'],
               "acl": userPrincipalsResponse['streamerInfo']['acl'] }

userPrincipalsResponse

In [ ]:
import selenium

driver = selenium.webdriver.Chrome()

In [ ]:
tda.auth.client_from_login_flow(driver, 
                                api_key=config.TDAMERITRADE, 
                                redirect_url='https://127.0.0.1', 
                                token_path='./tmp_token/token.pickle')

In [ ]:
from tda.auth import easy_client
from tda.client import Client
import config

c = easy_client(
        api_key=config.TDAMERITRADE,
        redirect_uri='https://127.0.0.1',
        token_path='./tmp_token/token2.pickle')

resp = c.get_price_history('AAPL',
        period_type=Client.PriceHistory.PeriodType.YEAR,
        period=Client.PriceHistory.Period.TWENTY_YEARS,
        frequency_type=Client.PriceHistory.FrequencyType.DAILY,
        frequency=Client.PriceHistory.Frequency.DAILY)
assert resp.ok
history = resp.json()


In [ ]:
history